In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

# Import dataset and Create dataframe

## Train set

In [2]:
train_data_fol = '../data/Training Data/'

train_img_path = []
train_labels = []

train_label_fol = os.listdir(train_data_fol)

for fol in train_label_fol:
    fol_path = os.path.join(train_data_fol,fol)
    files = os.listdir(fol_path)
    for img in files:
        img_path = os.path.join(fol_path,img)
        train_img_path.append(img_path)
        train_labels.append(fol)

train_df = pd.DataFrame(data={'file_path':train_img_path,'label':train_labels})
        

In [43]:
train_df
# 2000 images,15 classes 
# total should have 30000 observations

,file_path,label
0,../data/Training Data/Beetle\Beetle-Train (1)....,Beetle
1,../data/Training Data/Beetle\Beetle-Train (1).jpg,Beetle
2,../data/Training Data/Beetle\Beetle-Train (10)...,Beetle
3,../data/Training Data/Beetle\Beetle-Train (10)...,Beetle
4,../data/Training Data/Beetle\Beetle-Train (100...,Beetle
...,...,...
29995,../data/Training Data/Zebra\Zebra (995).jpeg,Zebra
29996,../data/Training Data/Zebra\Zebra (996).jpeg,Zebra
29997,../data/Training Data/Zebra\Zebra (997).jpeg,Zebra
29998,../data/Training Data/Zebra\Zebra (998).jpeg,Zebra


In [44]:
os.getcwd()

'c:\\Users\\Quah\\OneDrive\\Desktop\\New folder\\Convolutional_Neural_Network\\src'

In [45]:
train_label_fol

['Beetle',
 'Butterfly',
 'Cat',
 'Cow',
 'Dog',
 'Elephant',
 'Gorilla',
 'Hippo',
 'Lizard',
 'Monkey',
 'Mouse',
 'Panda',
 'Spider',
 'Tiger',
 'Zebra']

## Validation set

In [3]:
val_data_fol = '../data/Validation Data/'

val_img_path = []
val_labels = []

val_label_fol = os.listdir(val_data_fol)

for fol in val_label_fol:
    fol_path = os.path.join(val_data_fol,fol)
    files = os.listdir(fol_path)
    for img in files:
        img_path = os.path.join(fol_path,img)
        val_img_path.append(img_path)
        val_labels.append(fol)

val_df = pd.DataFrame(data={'file_path':val_img_path,'label':val_labels})

In [47]:
val_df

,file_path,label
0,../data/Validation Data/Beetle\Beatle-Valid (1...,Beetle
1,../data/Validation Data/Beetle\Beatle-Valid (1...,Beetle
2,../data/Validation Data/Beetle\Beatle-Valid (1...,Beetle
3,../data/Validation Data/Beetle\Beatle-Valid (1...,Beetle
4,../data/Validation Data/Beetle\Beatle-Valid (1...,Beetle
...,...,...
1995,../data/Validation Data/Zebra\Zebra-Valid (366...,Zebra
1996,../data/Validation Data/Zebra\Zebra-Valid (367...,Zebra
1997,../data/Validation Data/Zebra\Zebra-Valid (368...,Zebra
1998,../data/Validation Data/Zebra\Zebra-Valid (369...,Zebra


## Test set

In [4]:
test_data_fol = '../data/Testing Data/'

test_img_path = []
test_labels = []

test_label_fol = os.listdir(test_data_fol)

for fol in test_label_fol:
    fol_path = os.path.join(test_data_fol,fol)
    files = os.listdir(fol_path)
    for img in files:
        img_path = os.path.join(fol_path,img)
        test_img_path.append(img_path)
        test_labels.append(fol)

test_df = pd.DataFrame(data={'file_path':test_img_path,'label':test_labels})

In [49]:
test_df

,file_path,label
0,../data/Testing Data/Beetle\Beatle-Test (1).jpeg,Beetle
1,../data/Testing Data/Beetle\Beatle-Test (1).jpg,Beetle
2,../data/Testing Data/Beetle\Beatle-Test (10).jpeg,Beetle
3,../data/Testing Data/Beetle\Beatle-Test (10).jpg,Beetle
4,../data/Testing Data/Beetle\Beatle-Test (11).jpeg,Beetle
...,...,...
2402,../data/Testing Data/Zebra\Zebra-Test (95).jpeg,Zebra
2403,../data/Testing Data/Zebra\Zebra-Test (96).jpeg,Zebra
2404,../data/Testing Data/Zebra\Zebra-Test (97).jpeg,Zebra
2405,../data/Testing Data/Zebra\Zebra-Test (98).jpeg,Zebra


# Data Preprocessing

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tr_gen = ImageDataGenerator(horizontal_flip=True)

train_gen = tr_gen.flow_from_dataframe(train_df,x_col='file_path',y_col='label',
                                    target_size=(224,224),color_mode='rgb',
                                    class_mode='categorical',batch_szie=64)

Found 30000 validated image filenames belonging to 15 classes.


In [6]:
val_test_gen = ImageDataGenerator()
val_gen = val_test_gen.flow_from_dataframe(val_df,x_col = 'file_path',y_col = 'label',
                                           target_size=(224,224),color_mode='rgb',
                                           class_mode='categorical',batch_size=64)

test_gen = val_test_gen.flow_from_dataframe(test_df,x_col = 'file_path',y_col = 'label',
                                           target_size=(224,224),color_mode='rgb',
                                           class_mode='categorical',batch_size=64)


Found 2000 validated image filenames belonging to 15 classes.
Found 2407 validated image filenames belonging to 15 classes.


# Build the model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adamax

In [8]:
base_model = tf.keras.applications.EfficientNetV2B1(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3),
    pooling = 'max'
)

In [9]:
model = Sequential([base_model,
                    BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001),
                    Dense(256,activation='relu'),
                    Dropout(rate=0.45,seed=123),
                    Dense(15,activation='softmax')])

model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b1 (Functional)  │ (None, 1280)           │     6,931,124 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,268,035 (27.73 MB)

 Trainable params: 7,194,403 (27.44 MB)

 Non-trainable params: 73,632 (287.62 KB)

In [11]:
history = model.fit(train_gen, epochs=40, 
                    validation_data=val_gen, batch_size=64)

Epoch 1/40
110/938 ━━━━━━━━━━━━━━━━━━━━ 50:11 4s/step - accuracy: 0.6377 - loss: 1.2685

KeyboardInterrupt: 

In [ ]:
plot_training(history)